In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-10-21 16:00:16--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  58.8MB/s    in 0.4s    

2022-10-21 16:00:17 (58.8 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
# Mutate count columns
df_ratings['count_users'] = df_ratings.groupby(['user'], group_keys=False).transform('count')['rating']
df_ratings['count_books'] = df_ratings.groupby(['isbn'], group_keys=False).transform('count')['rating']

# Clean data
df_ratings = df_ratings.loc[(df_ratings.count_users >= 200) & (df_ratings.count_books >= 100)]
df_ratings = df_ratings[['user','isbn','rating']]

# Merge dataframes
df = pd.merge(df_ratings, df_books, on="isbn")

# Remove duplicates (Same user several reviews over same book)
df = df.drop_duplicates(subset=['user','title'])

# Transpose dataframe to have tittle vs users with values in each cell. Introduce 0's for NAs.
dfknn = df.pivot(index='title', columns='user', values='rating').fillna(0)

dfknn.shape

# Get list of titles (Books)
titles = list(dfknn.index.values)

# Functions to transform title to index and get row values for the title
def title_2_index(data, title, titles=titles):
  ind = titles.index(title)
  return ind, data.values[ind,:]

def index_2_title(ind, titles=titles):
  return titles[ind]

# Examples modify title to index
x, y = title_2_index(dfknn, '1984')
print('Index:', x)
print('First 10 values in the row:', y[1:10])
print('Book name from index:', index_2_title(x))

# Model (Euclidean distance - p=2)
neigh = NearestNeighbors(metric="cosine",algorithm="brute")
neigh.fit(dfknn.values)

Index: 0
First 10 values in the row: [0. 0. 0. 0. 0. 0. 0. 0. 0.]
Book name from index: 1984


NearestNeighbors(algorithm='brute', metric='cosine')

In [5]:
# function to return recommended books - this will be tested
def get_recommends(book = "", titles=titles, neighbors=5):
  ind, val = title_2_index(dfknn, book)
  pred = neigh.kneighbors(np.reshape(val,[1,-1]),len(titles),return_distance=True)
  # I give all the list but extract the last 5 (Highest probability)
  prob_val = pred[0][0][1:neighbors+1]
  prob_ind = pred[1][0][1:neighbors+1]
  pred = []
  for num in range(len(prob_val)):
    pred.append([index_2_title(prob_ind[num]), prob_val[num]])
  # Create output
  pred = pred[::-1]
  recommended_books = [book, pred]
  return recommended_books

get_recommends('1984')


['1984',
 [["The Hitchhiker's Guide to the Galaxy", 0.77445555],
  ["Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death",
   0.77405274],
  ['The Catcher in the Rye', 0.76593226],
  ['Animal Farm', 0.7618247],
  ["The Handmaid's Tale", 0.75083876]]]

In [6]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
